The current code does not use a classifier; it directly calculates participants’ knowledge progression based on their pre-test and post-test scores. This approach is more of a rule-based classification rather than using machine learning.

How It Works:
	•	Computes the total score difference for each participant.
	•	Applies a rule-based function to classify them into three groups:
	•	"Improved" → If post-test score is higher than pre-test.
	•	"Declined" → If post-test score is lower than pre-test.
	•	"Same Stage" → If post-test score did not change.

This is a deterministic approach based on numeric differences, not a machine learning classifier.


YOU can keep only labels , make them numeric and based on this data , create clusters